In [ ]:
# get data
import pandas as pd
# ML stuff
from mne.decoding import CSP
from mne_bids import BIDSPath, read_raw_bids
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
# series to series transformer
from sktime.transformations.series.detrend import Detrender

# panel to panel transformers
from sktime_neuro.transformations.panel.baselinecorrection import \
    BaselineCorrectionTransformer as Baseline
from sktime_neuro.transformations.series.filterforseries import FilterforSeries
from sktime_neuro.transformations.series.seriesdownsampling import \
    SeriesDownsampling
from sktime_neuro.transformations.series_to_panel.eeg_epoching import epoch
# epoching
from sktime_neuro.utils import mne_processing as utils

## Get Bids Data into sktime Series format

In [ ]:
# example can be downloaded here: https://osf.io/cj2dr/
bids_path = BIDSPath(
    subject="08", task="matchingpennies", suffix="eeg", datatype="eeg", root="example"
)

raw = read_raw_bids(bids_path=bids_path, verbose=False)
data = raw.get_data()

## Apply Series to Series Transformer

In [ ]:
downsampled_data = SeriesDownsampling(2).fit_transform(data)

In [ ]:
detrended_data = Detrender().fit_transform(pd.DataFrame(downsampled_data.transpose()))

In [ ]:
filtered_data = FilterforSeries(l_freq=8, h_freq=16, sfreq=250).fit_transform(
    detrended_data.transpose()
)

## Epoch

In [ ]:
annotation = utils.create_annotation(raw)

data, labels = epoch(
    Z=filtered_data,
    annotation=annotation,
    labels=["right/2", "left/1"],
    interval=[-0.5, 1.5],
    sfreq=250,
)

## Apply Panel-To-Panel Transformers

In [ ]:
bl_corrected = Baseline(upper=0.5, fs=250).fit_transform(data)

## Classify Trials

In [ ]:
# Assemble a classifier

# filterer = PaneltoPanelFilter() # should not be done for short samples
csp = CSP(transform_into="average_power")
lda = LinearDiscriminantAnalysis()


param_grid = {
    "CSP__n_components": [1, 2, 3, 4, 5, 7, 10],
}

# Use scikit-learn Pipeline with cross_val_score function
pipe = Pipeline(steps=[("CSP", csp), ("LDA", lda)])

search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(bl_corrected, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)